In [25]:
import sys
sys.path.append('../RecSysRep/')

In [26]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=9123)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13645 ( 0.0%) Users that have less than 1 test interactions


In [14]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [27]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender

In [29]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = SLIMElasticNetRecommender(URM_train)

model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
model_init(recommender2, 'test', {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009})

../models_temp/Similarity_Hybrid/SLIMER.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIMER'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/test.zip
Model does not exists, creating...
SLIMElasticNetRecommender: Processed 1235 ( 6.8%) in 5.00 min. Items per second: 4.11
SLIMElasticNetRecommender: Processed 2972 (16.5%) in 10.00 min. Items per second: 4.95
SLIMElasticNetRecommender: Processed 4690 (26.0%) in 15.00 min. Items per second: 5.21
SLIMElasticNetRecommender: Processed 6426 (35.6%) in 20.01 min. Items per second: 5.35
SLIMElasticNetRecommender: Processed 8178 (45.3%) in 25.01 min. Items per second: 5.45
SLIMElasticNetRecommender: Processed 9913 (54.9%) in 30.01 min. Items per second: 5.50
SLIMElasticNetRecommender: Processed 11523 (63.8%) in 35.01 min. Items per second: 5.48
SLIMElasticNetRecommender: Processed 13199 (73.1%) in 40.01 min. Items per second: 5.50
SLIMElasticNetRecommender: Proce

In [18]:
recommender1_1 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)

model_init(recommender1_1, 'SLIM_all', models_to_combine_best['SLIM_all'])

../models_temp/Similarity_Hybrid/SLIM_all.zip
Model does not exists, creating...
(13989, 18059)


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [17:46<00:00, 16.93it/s]


SLIMElasticNetRecommender: Saving model in file '../models_temp/Similarity_Hybrid/SLIM_all'
SLIMElasticNetRecommender: Saving complete


In [19]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [30]:
rank_models(evaluator_validation, [recommender1, recommender2])

EvaluatorHoldout: Processed 13645 (100.0%) in 26.47 sec. Users per second: 515
            MAP
cutoff         
10      0.24782
SLIMElasticNetRecommender 0.2478195949443658
EvaluatorHoldout: Processed 13645 (100.0%) in 25.21 sec. Users per second: 541
             MAP
cutoff          
10      0.247839
SLIMElasticNetRecommender 0.24783942894367944


In [ ]:
class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        alpha = trial.suggest_uniform('alpha', 0, 1)
        beta = trial.suggest_uniform('beta', 0, 1)
        gamma = trial.suggest_uniform('gamma', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(2, alpha, beta, gamma)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            penalty = 0.245 + map_v
            trial.report(penalty, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender1, recommender2, recommender3, evaluator_validation), n_trials=500)

print(study.best_params)

In [24]:
import numpy as np

def prepare_submission(users_to_recommend: np.array, use_to_recommend: np.array, urm_train: sps.csr_matrix, recommender: object):

    recommendation_length = 10
    submission = []
    for idx in use_to_recommend:
        user_id = idx
        mapped_user_id = idx
        
        recommendations = recommender.recommend(user_id_array = mapped_user_id,
                                                cutoff=recommendation_length,
                                                remove_seen_flag=True)
        
        submission.append((user_id, [item_id for item_id in recommendations]))
        
    return submission 

URM_all_dataframe, _, _, _, _ = ld.getDataframes()
users_to_recommend = URM_all_dataframe["UserID"].unique()
submission = prepare_submission(URM_all_dataframe, users_to_recommend, URM_all, recommender1)

def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")

write_submission(submission)